In [1]:
from datasets import load_dataset
#dataset = load_dataset("rajpurkar/squad")
squad = load_dataset("squad")

/home/prasad/tf-gpu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import BertTokenizerFast, BertForQuestionAnswering
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
squad['train']

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 87599
})

In [4]:
def tokenize_function(example):
    return tokenizer(example["question"], example["context"], return_tensors="pt", padding=True)

tokenized_squad = squad.map(tokenize_function, batched=True)

Map: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 10570/10570 [00:01<00:00, 5985.89 examples/s]


In [5]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./squad_output",
    evaluation_strategy="steps",
    eval_steps=1000,
    save_steps=1000,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=3e-5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["validation"],
)

trainer.train()

2024-10-15 21:44:16.275477: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-15 21:44:16.282599: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-15 21:44:16.289683: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-15 21:44:16.291782: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-15 21:44:16.298067: I tensorflow/core/platform/cpu_feature_guar

RuntimeError: The size of tensor a (781) must match the size of tensor b (512) at non-singleton dimension 1